In [35]:
'''
The code developed in this Scenario is mainly based off the code already available in the
Flow tutorial notebooks. The purpose of this Scenario is to serve as a
benchmark for the I-210 subsection problem to be later used to train an agent.
'''
# Supress system warnings that have no effect
import warnings
warnings.filterwarnings('ignore')

# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams, InFlows, NetParams
from flow.core.params import InitialConfig, EnvParams, SumoParams

from flow.controllers import ContinuousRouter, IDMController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams

In [36]:
# Use this hand-in-hand with OSM files
from flow.scenarios import Scenario

In [37]:
# specify the sub-section as a scenario
'''
You're going to want to change the path of the OSM file to the path
that you have it saved under. Go through the following steps in order to render
the same west wing subsection of the I-210 Highway:
1) Visit the following link: https://www.openstreetmap.org/#map=17/34.11926/-117.82398
2) Click on the 'Export' button on the top left corner of the screen
3) Click the 'Export' button in blue that showed up in the pop-up menu
   on the left side of the screen
4) Set the 'osm_path' parameter below to 'path/to/west_wing.osm'

**Note you may have to rename the file, if so make sure to specify the .osm in the file name.
'''
net_params = NetParams(
    osm_path='/Users/apple/Desktop/Berkeley/Forks/i210/west_wing.osm',
    no_internal_links=False
)

In [38]:
# create the remainding parameters
env_params = EnvParams()
sim_params = SumoParams(render=True, emission_path='/Users/apple/Desktop/Berkeley/Forks/flow/emissions/')
initial_config = InitialConfig(spacing="random", perturbation=2)
vehicles = VehicleParams()

vehicles.add(
    veh_id = "human",
    acceleration_controller=(IDMController, {}),
    routing_controller = (ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode="obey_safe_speed",
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode= "strategic",
    ), num_vehicles = 400)

# create the scenario
scenario = Scenario(
    name='west_wing',
    net_params=net_params,
    initial_config=initial_config,
    vehicles=vehicles
)

In [39]:
# create the environment
env = TestEnv(
    env_params=env_params,
    sim_params=sim_params,
    scenario=scenario
)

# run the simulation for 1000 steps
exp = Experiment(env=env)
exp.run(1, 10)

No routes specified, defaulting to single edge routes.
Round 0, return: 0
Average, std return: 0.0, 0.0
Average, std speed: 0.5330237850882228, 0.0


{'mean_returns': [0.0],
 'per_step_returns': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 'returns': [0],
 'velocities': [array([0.09695235, 0.1938753 , 0.29077632, 0.38766221, 0.48454275,
         0.58143332, 0.67834922, 0.7752518 , 0.87222225, 0.96917233])]}

In [40]:
'''
Up until this point we have Human cars originating on all edges.
Let us limit them to the highway strip belonging to the I-210 as well as it's round semi-circle
section by defining the 'EDGES_DISTRIBUTION' variable in the InitialConfig().
'''

EDGES_DISTRIBUTION = [
    # right semi circle
    "182793343",
    "29407664",
    "51139309",
    "51139310#0",
    "51139310#1",
    "156471827",
    "15647183",

    # left semi circle   
    "29407755",
    "48204049",
    
    # top section of I-210, heading east
    "156471837",
    "27260026#0",
    
    # ----
    "302013371",
    
    "156471822"
]

# the above variable is added to initial_config
new_initial_config = InitialConfig(
    edges_distribution=EDGES_DISTRIBUTION
)

In [41]:
import random

edge_options = {
            # right semi circle
            "182793343" : ["182793343","29407664", "51139309", 
                           "51139310#0", "51139310#1", "156471827", "156471813"],
            
            "29407664" : ["29407664", "51139309", "51139310#0", "51139310#1", "156471827", 
                          "156471813"],
            
            "51139309" : ["51139309", "51139310#0", "51139310#1", "156471827", "156471813"],
            
            "51139310#0" : ["51139310#0", "51139310#1", "156471827", "156471813"],
            
            "51139310#1" : ["51139310#1", "156471827", "156471813"],
            
            "156471827" : ["156471827", "156471813"],
            
            "156471813" : ["156471813"],
            
            # left semi circle
            "29407755" : ["29407755", "48204049"],
            
            "48204049" : ["48204049"],
            
            # top section heading east
            "156471837" : ["156471837", "27260026#0"],
            
            "27260026#0" : ["27260026#0"],
            
            # ----
            "302013371" : ["302013371"],
            
            "156471822" : ["156471822"]
        }

class InterstateSubSection(Scenario):
    '''
    User defined class in order to allow continuous driving for vehicles on custom edges.
    
    This class derives the basic functionality of the Scenario class as well as defining
    as function to create continuous routing for vehicles on user defined edges.
    
    Function(s):
    -----------
    specify_routes: 
                Returns a tuple of dictionaries where the keys correspond to the edges defined
                earlier in the 'EDGES_DISTRIBUTION' variable, and the values are the remaining 
                edges the follow the key.
    '''
    def specify_routes(self, net_params):
#         n = random.randint(0,2)
#         if (n == 0):
#             return edge_option_1
        return edge_options

In [42]:
inflow = InFlows()

# Inflow of cars on main East-heading highway
inflow.add(veh_type="human",
           edge="156471837",
           vehs_per_hour=5000,
           departSpeed=10,
           departLane="random")

# Inflow of cars on right semi circle
inflow.add(veh_type="human",
           edge="182793343",
           vehs_per_hour=1000,   # 3000,
           departSpeed=10,
           departLane="random")

inflow_net_params = NetParams(
    osm_path='/Users/apple/Desktop/Berkeley/Forks/i210/west_wing.osm',
    no_internal_links=False,
    inflows=inflow
)

In [44]:
# create the scenario
new_scenario = InterstateSubSection(
    name='west_wing',
    net_params=inflow_net_params,
    initial_config=new_initial_config,
    vehicles=vehicles,
)

# create the environment
env = TestEnv(
    env_params=env_params,
    sim_params=sim_params,
    scenario=new_scenario
)

# run the simulation for 1000 steps
exp = Experiment(env=env)
exp.run(1, 2500)

KeyboardInterrupt: 